# Analisis de Sentimientos en reseñas de Steam.
**Nombre**
**Fecha**
**Materia**

## Descripcion
> text

## Importacion de librerias


Aquí se importan las librerías esenciales para el procesamiento de lenguaje natural, análisis de datos y visualización. Utilizaremos nltk para el análisis de texto, pandas para la manipulación de dataframes, matplotlib y seaborn para graficar, y otras librerías como re y random para tareas específicas.

- `nltk` : Para el analisis de texto
- `pandas` : Para Para la manipulacion de dataframes
- `matplotlib y seaborn` : Para graficar
- `re` : Para expresiones regulares

In [1]:
# Importación de librerías para Procesamiento de Lenguaje Natural
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
import random
import requests

from nltk.corpus import stopwords
from nltk import ngrams

# Descargar recursos de nltk, como stopwords
#nltk.download('punkt')
#nltk.download('stopwords')

# Confirmación de importación
print("Librerías importadas correctamente para el procesamiento de lenguaje natural.")

Librerías importadas correctamente para el procesamiento de lenguaje natural.


## Importacion de Datos de Steam
En esta parte se explica como obtener las reseñas de Steam. Se pueden extraer mediante la API de Steam, utilizando el siguiente endpoint:

`GET https://store.steampowered.com/appreviews/<id_de_aplicación>?json=1`
Se indican algunos parametros utiles como lo son:
- `filter`
- `language`
- `day_range`
- `cursor`
- `review_type`
- `num_per_page`
- `filter_offtopic_activity`

Para saber mas de los parametros puedes ir a [Steam API](https://partner.steamgames.com/doc/store/getreviews)

En esta parte se hace la peticion del endpoint mediante el `id del juego` con los parametros adecuados (En este caso, se extraen 100 reseñas por pagina en lenguaje de español).
La respuesta se presenta en estructura JSON devuelto y psoteriormente se carga en un DataFrame de *Pandas* para hacer el análisis

In [28]:
# URL de la API de Steam para obtener reseñas en español (100 reseñas)
id_game = "2246340"             # Monster Hunter Wilds = 2246340
language = "spanish"            # Idioma de las reseñas
num_per_page = "100"            # Número de reseñas por página
url = f"https://store.steampowered.com/appreviews/{id_game}?json=1&language={language}&num_per_page={num_per_page}"


# Realizar la solicitud a la API
response = requests.get(url)
data = response.json()

# Extraer las reseñas del JSON
reviews = data.get('reviews', [])

# Crear un DataFrame a partir de las reseñas
df_reviews = pd.DataFrame(reviews)

# Mostrar información básica del DataFrame
print("Información del DataFrame:")
print(df_reviews.info())


Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   recommendationid             100 non-null    object
 1   author                       100 non-null    object
 2   language                     100 non-null    object
 3   review                       100 non-null    object
 4   timestamp_created            100 non-null    int64 
 5   timestamp_updated            100 non-null    int64 
 6   voted_up                     100 non-null    bool  
 7   votes_up                     100 non-null    int64 
 8   votes_funny                  100 non-null    int64 
 9   weighted_vote_score          100 non-null    object
 10  comment_count                100 non-null    int64 
 11  steam_purchase               100 non-null    bool  
 12  received_for_free            100 non-null    bool  
 13  written_d

In [27]:
print("\nPrimeras filas del DataFrame completo:")
df_reviews.head(5)


Primeras filas del DataFrame completo:


,recommendationid,author,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,primarily_steam_deck
0,189321117,"{'steamid': '76561198138131071', 'num_games_ow...",spanish,"AMO el juego, es demasiado bueno y la verdad e...",1741064282,1741840734,False,135,6,0.832816362380981445,0,True,False,False,False
1,188940182,"{'steamid': '76561198049254557', 'num_games_ow...",spanish,Basta ya. Basta de sacar productos incompletos...,1740743526,1740743526,False,79,3,0.70836329460144043,0,True,False,False,False
2,189266847,"{'steamid': '76561198066270836', 'num_games_ow...",spanish,Uno de los mejores juegos de la historia sin d...,1741010357,1741010357,True,23,2,0.706308245658874512,1,True,False,False,False
3,189254007,"{'steamid': '76561198010057144', 'num_games_ow...",spanish,"As a Monster Hunter game, it's absolutely supe...",1740997022,1740997022,False,48,3,0.693980157375335693,5,True,False,False,False
4,188929055,"{'steamid': '76561198417230907', 'num_games_ow...",spanish,el juego de la banana esta mejor optimizado \n...,1740735215,1740751024,True,30,0,0.71985316276550293,0,True,False,False,False


In [30]:
# Crear otro dataframe para unicamente la columna de review y salga completamente el texto
df_reviewsComments = df_reviews[['review']]
df_reviewsComments.head(5)

,review
0,"AMO el juego, es demasiado bueno y la verdad es que superó todas mi expectativas,\nPERO, tiene una de las peores optimizaciones que he visto en mi vida, necesita parchearse URGENTE"
1,Basta ya. Basta de sacar productos incompletos. Si el juego va mal en PCs de gama media (y también en muchos equipos de alta gama) se retrasa el juego. Hoy en día es un estándar en la industria pero debemos dejar de tragar los usuarios. Le voy a dar un No Recomendado hasta que arreglen el juego porque ahora mismo es injugable. \nUna pena la verdad porque por lo poco que he visto parece que el juego promete mucho pero es que tienen que dejar de abusar del consumidor y nuestra manera de quejarnos es influenciar en sus ventas con comentarios negativos.
2,"Uno de los mejores juegos de la historia sin duda. Puro gameplay, pura diversión, sin chorradas. Entiendo que a algunos les vaya mal. Yo personalmente tengo una 2070 super, y quitandole las tonterias del desenfoque de movimiento, bajando un poco las sombras y un par de ajustes puedo tener las texturas en alto sin problema, 60 fps"
3,"As a Monster Hunter game, it's absolutely superb, delicious, amazing. It's everything I've always loved about the Monster Hunter franchise and more. And I've been playing since the PSP era with Monster Hunter Freedom Unite.\r\n\r\nHowever, needing frame gen only to reach 60 FPS at 1080p on a 3060ti (RTX 2080 equivalent, two tiers above recommended minimum) and Ryzen 9 5900x is absolutely unacceptable. Fix the performance up to an acceptable level and this review will become positive. The day I can run it at medium graphics, 1080p 60-70 stable fps with no scaling tech, I'll consider myself satisfied."
4,el juego de la banana esta mejor optimizado \nparte 2\ngente.. me di cuenta que la opción del juego (generación de fotogramas) me va como el culo así que por curiosear ingrese a la aplicación de AMD (mi GPU 6600XT) y active el AMD Fluid Motion Frames en graficos altos y el juego aparenta ir a 100 FPS de media 80.


## Limpieza de texto (Stopwords)
En esta seccion se muestra como limpiar el texto de las reseñas, removiendo stopwords en español para preparar el analisis. Se utilizan las stopwords de *nltk*


In [ ]:
# Ejemplo de URL y parámetros para obtener reseñas desde la API de Steam
app_id = "2246340"  # Ejemplo: Monster Hunter Wilds
url = f"https://store.steampowered.com/appreviews/{app_id}?json=1&language=spanish"

# Se pueden agregar más parámetros según la necesidad:
# params = {
#     "filter": "recent",
#     "language": "spanish",
#     "day_range": 30,
#     "cursor": "*",
#     "review_type": "all",
#     "num_per_page": 100,
#     "filter_offtopic_activity": 0
# }

# Simulación de la extracción de datos (en la práctica, utilizarías requests o kaggle API para obtener el JSON)
# Por ejemplo:
# response = requests.get(url, params=params)
# data = response.json()
# df_reviews = pd.DataFrame(data['reviews'])

# Para la simulación, creamos un dataframe vacío
df_reviews = pd.DataFrame()  
print("Simulación: Datos de reseñas importados correctamente.")


Simulación: Datos de reseñas importados correctamente.


In [5]:
# Descargar el dataset de reseñas de Steam desde Kaggle
# Asegúrate de tener configurado el API de Kaggle (archivo kaggle.json en ~/.kaggle/)
!kaggle datasets download -d andrewmvd/steam-reviews-data

# Descomprimir el dataset descargado (se sobreescriben archivos existentes si los hay)
!unzip -o steam-reviews-data.zip

# Cargar el dataset en un DataFrame de pandas
import pandas as pd

# Asumimos que el archivo descomprimido se llama 'steam_reviews.csv'
df = pd.read_csv('steam_reviews.csv')

# Mostrar las primeras filas para entender la estructura del DataFrame
print("Primeras filas del DataFrame:")
print(df.head())

# Filtrar las reseñas del juego con ID 2246340 (Monster Hunter: Wilds)
# Se asume que la columna 'appid' almacena el ID de cada juego
app_id = 2246340
df_game = df[df['appid'] == app_id]

# Si existe una columna 'language', filtramos las reseñas en español
if 'language' in df_game.columns:
    df_game = df_game[df_game['language'] == 'spanish']

# Validación: Mostrar información básica y número de reseñas importadas
print("\nInformación del DataFrame para el juego con ID", app_id)
print(df_game.info())
print("\nNúmero de reseñas importadas:", df_game.shape[0])


"kaggle" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"unzip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


FileNotFoundError: [Errno 2] No such file or directory: 'steam_reviews.csv'

Validacion del DataFrame

In [4]:
# Mostrar información básica del dataframe (columnas, filas, encabezados)
print("Información del DataFrame de reseñas:")
print(df_reviews.info())


Información del DataFrame de reseñas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame
None


In [ ]:
#Librerias
#import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
#import os

#from skimage import io
#from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import OneHotEncoder
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score
#from skimage.transform import resize

# Procesamiento de lenguaje Natural
#import nltk

#Importación de la Librería TensorFlow
#import tensorflow as tf
#from tensorflow.keras.layers import Dense
#from tensorflow.keras.callbacks import TensorBoard
#from keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
